In [162]:
import pkg_resources
from symspellpy.symspellpy import SymSpell, Verbosity

In [163]:
# import json
# import string
# import re

# with open('dataset/1-gram.json') as fopen:
#     myfile = json.load(fopen)

# with open("dataset/malay-words.txt", "w", encoding="utf-8") as text_file:
#     j=1
#     for i in myfile:
#         # n = i.strip(string.punctuation)
#         j+=1
#         n = re.sub(r'[^\w\s]','',i)
#         if j >= 248787:
#             n = n.rstrip("\n")

#         text_file.write(f'{n} {myfile[i]}\n')

In [164]:
# import json
# import string
# import re
# from collections import Counter

# with open('dataset/dumping-wiki-6-july-2019.json') as fopen:
#     myfile = json.load(fopen)

# corpus = []
# with open("dataset/malay-words.txt", "w", encoding="utf-8") as text_file:
#     for line in myfile:
#         line = line.replace('\n', ' ').replace('\t', ' ').lower()
#         line = re.sub('[^a-z ]', ' ', line)
#         text_file.write(f'{line}\n')
#     # tokens = line.split(' ')
#     # tokens = [token for token in tokens if len(token) > 0]
#     # corpus.extend(tokens)

# # corpus = Counter(corpus)


In [165]:
# from symspellpy import SymSpell

# sym_spell = SymSpell()
# corpus_path = 'dataset/malay-words.txt'
# sym_spell.create_dictionary(corpus_path)

# print(sym_spell.words)

In [166]:
# # build symspell tree 
# sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
# #loading dictionary/lexicon
# dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

In [167]:
# # term_index is the column of the term and count_index is the
# # column of the term frequency
# sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# # lookup suggestions for single-word input strings
# input_term = "sms"  # misspelling of "members"
# # max edit distance per lookup
# print("Misspelled Word - ",input_term)
# # (max_edit_distance_lookup <= max_dictionary_edit_distance)
# suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST,
#                                max_edit_distance=2)
# # display suggestion term, term frequency, and edit distance
# print("Best Candidate - ")
# for suggestion in suggestions:
#     print(suggestion)

In [168]:
# from dataset.word import words 
# with open("dataset/malay-words.txt", "w", encoding="utf-8") as text_file:
#     for w in words:
#         text_file.write(f'{w}\n')

In [198]:
with open('dataset/positive_words_ms.txt') as f:
    pos_words1 = f.readlines()

f.close()

for i in range(len(pos_words1)):
    pos_words1[i] = pos_words1[i].rstrip("\n")

In [199]:
with open('dataset/negative_words_ms.txt') as f:
    neg_words1 = f.readlines()

f.close()

for i in range(len(neg_words1)):
    neg_words1[i] = neg_words1[i].rstrip("\n")

In [200]:
import json

with open('dataset/sentiment.json') as fopen:
    myfile = json.load(fopen)

neg_words2 = myfile['negative']
pos_words2 = myfile['positive']

In [201]:
neg_words = np.unique(neg_words1+neg_words2)
pos_words = np.unique(pos_words1+pos_words2)

In [202]:
import numpy as np 
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import Counter
import string
import re
import stopwordsiso as sw

# download required library from nltk
# nltk.download('punkt')

# create stemmer for bahasa
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stop words consist of malay, indo, english
stop_words_main = list(sw.stopwords(["ms", "id", "en"]))
# custom stopwords such as shortform
stop_words_custom = ['kau', 'yg', 'mcm', 'gak', 'nak', 'ni', 'tu', 'la', 'je', 'kat', 'ya', 'dgn', 'tau', 'org', 'rt', 'aja', 'nk', 'dah',
                        'orang', 'sy', 'ga', 'kalo', 'kena']
STOP_WORDS = np.unique(stop_words_main+stop_words_custom)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def text_preprocessing(text):

    # remove numbers
    text = re.sub(r'\d+', '', text)

    # remove links
    text = re.sub('http[s]?://\S+', '', text)

    # remove word with tweethandle @name
    text = re.sub('[^ ]*@[^ ]*', '', text)

    # remove emoji
    text = remove_emoji(text)

    # tokennization
    tokens = word_tokenize(text)

    # remove punctuation
    words = []
    for token in tokens:
        if token not in string.punctuation:
            # temp = stemmer.stem(token)
            words.append(token)

    # # remove stopwords
    # cleaned = []
    # for word in words:
    #     if word not in STOP_WORDS:
    #         cleaned.append(word)

    # join all words into a complete sentence 
    complete_sentence = ' '.join([str(word) for word in words])

    # remove extra line spaces between words in a sentence
    complete_sentence = " ".join(complete_sentence.split())
    
    return complete_sentence

In [203]:
text = "Ironinya, penuntut universiti yang tanpa sebarang paksaan hadir ke program pro pembangkang dihalang, malah tindakan disiplin diambil."

text_preprocessing(text)

'Ironinya penuntut universiti yang tanpa sebarang paksaan hadir ke program pro pembangkang dihalang malah tindakan disiplin diambil'

In [204]:
def evaluate(text):

    tokens = word_tokenize(text)

    value = 0
    for token in tokens:
        if token in pos_words:
            value+=1
        elif token in neg_words:
            value+=-1
    
    if value > 0:
        result = 1
    elif value < 0:
        result = -1
    else:
        result = 0

    return result

In [205]:
evaluate(text)

-1

In [206]:
test_data1 = pd.read_csv('dataset/news-test-data.csv')
test_data1.loc[test_data1['label'] == 'Neutral', 'label'] = 0
test_data1.loc[test_data1['label'] == 'Positive', 'label'] = 1
test_data1.loc[test_data1['label'] == 'Negative', 'label'] = -1
test_data1.loc[test_data1['label'] == 'negative', 'label'] = -1
test_data1['label'] = test_data1['label'].astype('int64')
test_data1 = test_data1[['text', 'label']]
print(test_data1.head(10))
print(len(test_data1))

                                                text  label
0  Permohonan ini juga bertujuan memohon penjelas...      1
1  Projek ini bukti komitmen berterusan kerajaan ...      1
2  Menerusi pembentangan Bajet 2018 yang lalu, ke...      1
3  Berdasarkan kualiti jawapan calon, 27 peratus ...      0
4  Pada masa sama, kerajaan negeri juga telah bua...      1
5  Kita bawa pembangunan, bukan sahaja untuk Saba...      1
6  Harapan kita agar isu berkaitan golongan muda ...      1
7  Kesan daripada penyemakan ini antaranya membab...      0
8  Kalau dah bersara dan tiada kaitan lagi dengan...      1
9  Kesediaan negara-negara ITRC untuk mematuhi ko...      1
3688


In [207]:
text_cleaning = lambda x: text_preprocessing(x)
test_data1['cleaned_text'] = pd.DataFrame(test_data1['text'].apply(text_cleaning))
test_data1.head(10)

,text,label,cleaned_text
0,Permohonan ini juga bertujuan memohon penjelas...,1,Permohonan ini juga bertujuan memohon penjelas...
1,Projek ini bukti komitmen berterusan kerajaan ...,1,Projek ini bukti komitmen berterusan kerajaan ...
2,"Menerusi pembentangan Bajet 2018 yang lalu, ke...",1,Menerusi pembentangan Bajet yang lalu kerajaan...
3,"Berdasarkan kualiti jawapan calon, 27 peratus ...",0,Berdasarkan kualiti jawapan calon peratus calo...
4,"Pada masa sama, kerajaan negeri juga telah bua...",1,Pada masa sama kerajaan negeri juga telah buat...
5,"Kita bawa pembangunan, bukan sahaja untuk Saba...",1,Kita bawa pembangunan bukan sahaja untuk Sabah...
6,Harapan kita agar isu berkaitan golongan muda ...,1,Harapan kita agar isu berkaitan golongan muda ...
7,Kesan daripada penyemakan ini antaranya membab...,0,Kesan daripada penyemakan ini antaranya membab...
8,Kalau dah bersara dan tiada kaitan lagi dengan...,1,Kalau dah bersara dan tiada kaitan lagi dengan...
9,Kesediaan negara-negara ITRC untuk mematuhi ko...,1,Kesediaan negara-negara ITRC untuk mematuhi ko...


In [208]:
evaluator = lambda x: evaluate(x)
test_data1['result'] = pd.DataFrame(test_data1['cleaned_text'].apply(evaluator))
test_data1.head(10)

,text,label,cleaned_text,result
0,Permohonan ini juga bertujuan memohon penjelas...,1,Permohonan ini juga bertujuan memohon penjelas...,0
1,Projek ini bukti komitmen berterusan kerajaan ...,1,Projek ini bukti komitmen berterusan kerajaan ...,1
2,"Menerusi pembentangan Bajet 2018 yang lalu, ke...",1,Menerusi pembentangan Bajet yang lalu kerajaan...,1
3,"Berdasarkan kualiti jawapan calon, 27 peratus ...",0,Berdasarkan kualiti jawapan calon peratus calo...,-1
4,"Pada masa sama, kerajaan negeri juga telah bua...",1,Pada masa sama kerajaan negeri juga telah buat...,-1
5,"Kita bawa pembangunan, bukan sahaja untuk Saba...",1,Kita bawa pembangunan bukan sahaja untuk Sabah...,0
6,Harapan kita agar isu berkaitan golongan muda ...,1,Harapan kita agar isu berkaitan golongan muda ...,1
7,Kesan daripada penyemakan ini antaranya membab...,0,Kesan daripada penyemakan ini antaranya membab...,0
8,Kalau dah bersara dan tiada kaitan lagi dengan...,1,Kalau dah bersara dan tiada kaitan lagi dengan...,1
9,Kesediaan negara-negara ITRC untuk mematuhi ko...,1,Kesediaan negara-negara ITRC untuk mematuhi ko...,1


In [209]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, f1_score, recall_score

def sentiment_model_predict(data_test_target,data_prediction):
    conf_matrix = confusion_matrix(data_test_target,data_prediction)
    acc_score = accuracy_score(data_test_target, data_prediction)
    pre_score = precision_score(data_test_target, data_prediction, average="macro")
    re_score = recall_score(data_test_target, data_prediction, average="macro")
    f_score = f1_score(data_test_target, data_prediction, average="macro")

    print("Accuracy : "+str(round(acc_score*100,2)))
    print("Precision : "+str(round(pre_score*100,2)))
    print("Recall : "+str(round(re_score*100,2)))
    print("F1-Score :"+str(round(f_score*100,2)))
    print(conf_matrix)

In [210]:
actual = test_data1['label']
predict = test_data1['result']

sentiment_model_predict(actual,predict)

Accuracy : 47.1
Precision : 46.11
Recall : 47.48
F1-Score :44.93
[[468 245 181]
 [226 330 190]
 [566 543 939]]
